In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel("D:\BISI\level 2\CST2212\OPS-PM Baseline Cost (Synthetic).xlsx")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Order Number          64 non-null     int64  
 1   Change Request Title  64 non-null     object 
 2   Work Classification   64 non-null     object 
 3   Priority              64 non-null     object 
 4   Application Name      64 non-null     object 
 5   IT Directorate        64 non-null     object 
 6   IT Division           64 non-null     object 
 7   IT Section            64 non-null     object 
 8   Manager ID            64 non-null     int64  
 9   Effort Cost           64 non-null     float64
 10  Capacity to Deliver   64 non-null     object 
 11  Actuals               64 non-null     float64
dtypes: float64(2), int64(2), object(8)
memory usage: 6.1+ KB


In [4]:
df.isnull().sum()

Order Number            0
Change Request Title    0
Work Classification     0
Priority                0
Application Name        0
IT Directorate          0
IT Division             0
IT Section              0
Manager ID              0
Effort Cost             0
Capacity to Deliver     0
Actuals                 0
dtype: int64

In [5]:
df.duplicated()

0     False
1     False
2     False
3     False
4     False
      ...  
59    False
60    False
61     True
62    False
63    False
Length: 64, dtype: bool

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df['Order Number '] = df['Order Number '].astype(str)
df['Manager ID'] = df['Manager ID'].astype(str)

In [8]:
pd.crosstab(df['Priority'], df['Capacity to Deliver'])

Capacity to Deliver,No,Yes
Priority,,
Critical,0,2
High,0,5
Low,6,25
Medium,1,24


In [9]:
df.select_dtypes(include=['object']).nunique()

Order Number            13
Change Request Title    13
Work Classification      2
Priority                 4
Application Name        26
IT Directorate           6
IT Division             14
IT Section              31
Manager ID              27
Capacity to Deliver      2
dtype: int64

In [10]:
df.describe()

,Effort Cost,Actuals
count,63.000000,63.000000
mean,51831.532118,75166.872222
std,103950.113701,129514.417019
min,0.000000,0.000000
25%,29884.500000,13018.633333
50%,34576.000000,35320.680000
75%,39422.890137,70596.745346
max,815982.000000,634237.546746


In [11]:
df.corr()

C:\Users\zhuan\AppData\Local\Temp\ipykernel_41832\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Effort Cost,Actuals
Effort Cost,1.000000,0.646705
Actuals,0.646705,1.000000


In [13]:
df['Capacity to Deliver'].replace({'Yes': 1, 'No': 0}, inplace=True)
df['Capacity to Deliver']

0     1
1     1
2     1
3     1
4     1
     ..
58    1
59    1
60    1
62    1
63    1
Name: Capacity to Deliver, Length: 63, dtype: int64

In [14]:
df.head()

,Order Number,Change Request Title,Work Classification,Priority,Application Name,IT Directorate,IT Division,IT Section,Manager ID,Effort Cost,Capacity to Deliver,Actuals
0,161199,Sub-Project-4,Development,Low,Application A,DIR-1,Div-1,Sec-1,485317370,29414.500000,1,92798.636954
1,161454,Sub-Project-1,Development,Low,Application A,DIR-1,Div-1,Sec-1,485317370,149149.421875,1,470545.583046
2,161199,Sub-Project-4,Development,Low,Application A,DIR-1,Div-1,Sec-6,485317372,26029.000000,1,12603.025171
3,161277,Sub-Project-7,Enhancement,Low,Application A,DIR-1,Div-1,Sec-6,485317372,35841.599609,1,17354.204235
4,161454,Sub-Project-1,Development,Low,Application A,DIR-1,Div-1,Sec-6,485317372,30544.000000,1,14789.150594


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [16]:
df = df.drop('Change Request Title', axis=1)
df = df.drop('Actuals', axis=1)
X = df.drop('Capacity to Deliver', axis=1)
y = df['Capacity to Deliver']

In [17]:
# Label Encoding for 'Priority'
le = LabelEncoder()
X['Priority'] = le.fit_transform(X['Priority'])

In [18]:
# Frequency Encoding for 'Application Name', 'IT Division', 'IT Section', 'Manager ID'
for column in ['IT Division', 'IT Section', 'Application Name', 'Manager ID']:
    frequency_encoding = X[column].value_counts(normalize=True)
    X[column] = X[column].map(frequency_encoding)

In [19]:
# One-Hot Encoding for 'Order Number', 'Work Classification', 'IT Directorate'
# We can use pandas get_dummies or OneHotEncoder. Here, we will use get_dummies.
X = pd.get_dummies(X, columns=['Order Number ', 'Work Classification', 'IT Directorate'], drop_first=True)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [22]:
X_train['Effort Cost'] = scaler.fit_transform(X_train[['Effort Cost']])
X_test['Effort Cost'] = scaler.transform(X_test[['Effort Cost']])

In [23]:
df.isnull().sum()

Order Number           0
Work Classification    0
Priority               0
Application Name       0
IT Directorate         0
IT Division            0
IT Section             0
Manager ID             0
Effort Cost            0
Capacity to Deliver    0
dtype: int64

In [24]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred_train = logreg.predict(X_train)
y_pred = logreg.predict(X_test)

In [25]:
class_report_train = classification_report(y_train,y_pred_train)
conf_matrix_train = confusion_matrix(y_train,y_pred_train)

class_report = classification_report(y_test,y_pred)
conf_matrix = confusion_matrix(y_test,y_pred)

C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
print(class_report_train)
print(conf_matrix_train)

print(class_report)
print(conf_matrix)

              precision    recall  f1-score   support

           0       1.00      0.33      0.50         6
           1       0.92      1.00      0.96        44

    accuracy                           0.92        50
   macro avg       0.96      0.67      0.73        50
weighted avg       0.93      0.92      0.90        50

[[ 2  4]
 [ 0 44]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.92      1.00      0.96        12

    accuracy                           0.92        13
   macro avg       0.46      0.50      0.48        13
weighted avg       0.85      0.92      0.89        13

[[ 0  1]
 [ 0 12]]


In [27]:
cross_val_scores_accuracy = cross_val_score(logreg, X, y, cv=5, scoring='accuracy')
cross_val_scores_precision = cross_val_score(logreg, X, y, cv=5, scoring='precision')
cross_val_scores_recall = cross_val_score(logreg, X, y, cv=5, scoring='recall')
cross_val_scores_f1 = cross_val_score(logreg, X, y, cv=5, scoring='f1')
print('cross_val_scores_accuracy', cross_val_scores_accuracy)
print('cross_val_scores_accuracy_mean', cross_val_scores_accuracy.mean())
print('cross_val_scores_precision', cross_val_scores_precision)
print('cross_val_scores_precision_mean', cross_val_scores_precision.mean())
print('cross_val_scores_recall', cross_val_scores_recall)
print('cross_val_scores_recall_mean', cross_val_scores_recall.mean())
print('cross_val_scores_f1', cross_val_scores_f1)
print('cross_val_scores_recall_mean', cross_val_scores_f1.mean())

cross_val_scores_accuracy [1.         1.         0.92307692 0.83333333 1.        ]
cross_val_scores_accuracy_mean 0.9512820512820515
cross_val_scores_precision [1.         1.         0.91666667 1.         1.        ]
cross_val_scores_precision_mean 0.9833333333333332
cross_val_scores_recall [1.         1.         1.         0.81818182 1.        ]
cross_val_scores_recall_mean 0.9636363636363636
cross_val_scores_f1 [1.         1.         0.95652174 0.9        1.        ]
cross_val_scores_recall_mean 0.9713043478260868


In [28]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()

In [29]:
dtree.fit(X_train, y_train)

DecisionTreeClassifier()

In [30]:
y_pred_dtree_train = dtree.predict(X_train)
y_pred_dtree = dtree.predict(X_test)

print(classification_report(y_train,y_pred_dtree_train))
print(confusion_matrix(y_train,y_pred_dtree_train))
print(classification_report(y_test, y_pred_dtree))
print(confusion_matrix(y_test, y_pred_dtree))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        44

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

[[ 6  0]
 [ 0 44]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.92      1.00      0.96        12

    accuracy                           0.92        13
   macro avg       0.46      0.50      0.48        13
weighted avg       0.85      0.92      0.89        13

[[ 0  1]
 [ 0 12]]


C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
cross_val_scores_accuracy = cross_val_score(dtree, X, y, cv=5, scoring='accuracy')
cross_val_scores_precision = cross_val_score(dtree, X, y, cv=5, scoring='precision')
cross_val_scores_recall = cross_val_score(dtree, X, y, cv=5, scoring='recall')
cross_val_scores_f1 = cross_val_score(dtree, X, y, cv=5, scoring='f1')
print('cross_val_scores_accuracy', cross_val_scores_accuracy)
print('cross_val_scores_accuracy_mean', cross_val_scores_accuracy.mean())
print('cross_val_scores_precision', cross_val_scores_precision)
print('cross_val_scores_precision_mean', cross_val_scores_precision.mean())
print('cross_val_scores_recall', cross_val_scores_recall)
print('cross_val_scores_recall_mean', cross_val_scores_recall.mean())
print('cross_val_scores_f1', cross_val_scores_f1)
print('cross_val_scores_recall_mean', cross_val_scores_f1.mean())

cross_val_scores_accuracy [1.         0.92307692 0.92307692 0.75       0.58333333]
cross_val_scores_accuracy_mean 0.8358974358974359
cross_val_scores_precision [1.         0.91666667 0.91666667 1.         0.875     ]
cross_val_scores_precision_mean 0.9416666666666667
cross_val_scores_recall [1.         1.         1.         0.72727273 0.63636364]
cross_val_scores_recall_mean 0.8727272727272728
cross_val_scores_f1 [1.         0.95652174 0.95652174 0.9        0.8       ]
cross_val_scores_recall_mean 0.9226086956521738


In [33]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [34]:
rf.fit(X_train, y_train)

RandomForestClassifier()

In [35]:
y_pred_rf_train = rf.predict(X_train)
y_pred_rf = rf.predict(X_test)

print(classification_report(y_train,y_pred_rf_train))
print(confusion_matrix(y_train,y_pred_rf_train))
print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        44

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

[[ 6  0]
 [ 0 44]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.92      1.00      0.96        12

    accuracy                           0.92        13
   macro avg       0.46      0.50      0.48        13
weighted avg       0.85      0.92      0.89        13

[[ 0  1]
 [ 0 12]]


C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
cross_val_scores_accuracy = cross_val_score(rf, X, y, cv=5, scoring='accuracy')
cross_val_scores_precision = cross_val_score(rf, X, y, cv=5, scoring='precision')
cross_val_scores_recall = cross_val_score(rf, X, y, cv=5, scoring='recall')
cross_val_scores_f1 = cross_val_score(rf, X, y, cv=5, scoring='f1')
print('cross_val_scores_accuracy', cross_val_scores_accuracy)
print('cross_val_scores_accuracy_mean', cross_val_scores_accuracy.mean())
print('cross_val_scores_precision', cross_val_scores_precision)
print('cross_val_scores_precision_mean', cross_val_scores_precision.mean())
print('cross_val_scores_recall', cross_val_scores_recall)
print('cross_val_scores_recall_mean', cross_val_scores_recall.mean())
print('cross_val_scores_f1', cross_val_scores_f1)
print('cross_val_scores_recall_mean', cross_val_scores_f1.mean())

cross_val_scores_accuracy [1.         1.         0.92307692 0.83333333 0.91666667]
cross_val_scores_accuracy_mean 0.9346153846153846
cross_val_scores_precision [1.         1.         0.91666667 1.         0.91666667]
cross_val_scores_precision_mean 0.9666666666666666
cross_val_scores_recall [1.         1.         1.         0.81818182 1.        ]
cross_val_scores_recall_mean 0.9636363636363636
cross_val_scores_f1 [1.         1.         0.95652174 0.84210526 0.95652174]
cross_val_scores_recall_mean 0.9510297482837528


In [38]:
from sklearn.svm import SVC
svm = SVC()

In [39]:
svm.fit(X_train, y_train)

SVC()

In [40]:
y_pred_svm_train = svm.predict(X_train)
y_pred_svm = svm.predict(X_test)
print(classification_report(y_train,y_pred_svm_train))
print(confusion_matrix(y_train,y_pred_svm_train))
print(classification_report(y_test, y_pred_svm))
print(confusion_matrix(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       1.00      0.33      0.50         6
           1       0.92      1.00      0.96        44

    accuracy                           0.92        50
   macro avg       0.96      0.67      0.73        50
weighted avg       0.93      0.92      0.90        50

[[ 2  4]
 [ 0 44]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.92      1.00      0.96        12

    accuracy                           0.92        13
   macro avg       0.46      0.50      0.48        13
weighted avg       0.85      0.92      0.89        13

[[ 0  1]
 [ 0 12]]


C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zhuan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
cross_val_scores_accuracy = cross_val_score(svm, X, y, cv=5, scoring='accuracy')
cross_val_scores_precision = cross_val_score(svm, X, y, cv=5, scoring='precision')
cross_val_scores_recall = cross_val_score(svm, X, y, cv=5, scoring='recall')
cross_val_scores_f1 = cross_val_score(svm, X, y, cv=5, scoring='f1')
print('cross_val_scores_accuracy', cross_val_scores_accuracy)
print('cross_val_scores_accuracy_mean', cross_val_scores_accuracy.mean())
print('cross_val_scores_precision', cross_val_scores_precision)
print('cross_val_scores_precision_mean', cross_val_scores_precision.mean())
print('cross_val_scores_recall', cross_val_scores_recall)
print('cross_val_scores_recall_mean', cross_val_scores_recall.mean())
print('cross_val_scores_f1', cross_val_scores_f1)
print('cross_val_scores_recall_mean', cross_val_scores_f1.mean())

cross_val_scores_accuracy [0.92307692 0.84615385 0.84615385 0.91666667 1.        ]
cross_val_scores_accuracy_mean 0.9064102564102564
cross_val_scores_precision [0.92307692 0.84615385 0.84615385 0.91666667 1.        ]
cross_val_scores_precision_mean 0.9064102564102564
cross_val_scores_recall [1. 1. 1. 1. 1.]
cross_val_scores_recall_mean 1.0
cross_val_scores_f1 [0.96       0.91666667 0.91666667 0.95652174 1.        ]
cross_val_scores_recall_mean 0.9499710144927537
